In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
%matplotlib inline  
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os
import time

from StringIO import StringIO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import expon

import scipy.stats as ss
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
from sklearn import tree 
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import f1_score 
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestClassifier 
from sklearn.cluster import KMeans as KNN
from sklearn.neighbors import KNeighborsClassifier as KNeighborsClassifier
from sklearn.svm import SVC as SVC
from sklearn.svm import SVR as SVR 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB as GaussianN
from sklearn.lda import LDA 
from sklearn.qda import QDA  
from sklearn.tree import DecisionTreeClassifier as DecisionTreeClassifier

from sklearn.grid_search import GridSearchCV

#from sklearn.metrics.confusion_matrix import n

from sklearn.cluster import AgglomerativeClustering 
from sklearn import decomposition
from sklearn.preprocessing import label_binarize
from sklearn import svm
from sklearn.cluster import AgglomerativeClustering
from sklearn.tree import DecisionTreeClassifier as DecisionTreeClassifier
import seaborn as sns


from sknn.mlp import Classifier, Regressor, Layer
from sknn.mlp import Classifier, Layer, MultiLayerPerceptron

#os.getcwd()

C:\Users\brandon.tomlin\AppData\Local\Continuum\Anaconda\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\Users\brandon.tomlin\AppData\Local\Continuum\Anaconda\lib\site-packages\sklearn\qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)
C:\Users\brandon.tomlin\AppData\Local\Continuum\Anaconda\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implement

In [3]:
path = 'data/fertility_Diagnosis.txt'

names = ('Season', 
        'Age', 
        'Child_Diseases', 
        'Accidents', 
        'Surgical_Intervention',
        'Fever', 
        'Drinking', 
        'Smoking', 
        'Hours_Sitting', 
        'target')

df = pd.read_csv(path, names = names)

In [4]:
df.shape

(100, 10)

In [5]:
def perpareTarget(df, targetName, yes, no):   # prepares the Target column by mapping targetcolumn and moving to first col
    from sklearn import preprocessing
    le_dep = preprocessing.LabelEncoder()
    #to convert into numbers
    df[targetName] = le_dep.fit_transform(df[targetName])
    #df[targetName] = df[targetName].map({yes: '1', no: '0'})
    #df = df.fillna(0)
    targetSeries = df[targetName]
    del df[targetName]
    df.insert(0, targetName, targetSeries)
    df = df.rename(columns={targetName:'target'})
    expected = targetName
    df['target'] = df.target.astype(int)
    df = df.dropna()
    return df

targetName = ('target')  
df = perpareTarget(df, targetName, 'No', 'Yes')
df.head(2)

,target,Season,Age,Child_Diseases,Accidents,Surgical_Intervention,Fever,Drinking,Smoking,Hours_Sitting
0,0,-0.33,0.69,0,1,1,0,0.8,0,0.88
1,1,-0.33,0.94,1,0,1,0,0.8,1,0.31


## Setting feature testing and training
    second instance is of scaled data.
    Its important to note, that the dataset was already normalized before beginning

In [6]:
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing

features_train, features_test, target_train, target_test = train_test_split(df.ix[:,1:].values, 
                                                                            df.ix[:,0].values, 
                                                                            test_size=0.40, 
                                                                            random_state=0)

features_X_train, features_X_test, target_y_train, target_y_test = train_test_split(df.ix[:,1:].values, 
                                                                            df.ix[:,0].values, 
                                                                            test_size=.40, 
                                                                            random_state=0)
std_scale = preprocessing.StandardScaler().fit(features_X_train)
X_train = std_scale.transform(features_X_train)
X_test = std_scale.transform(features_X_test)

## Possible SMOTE required
    Synthetic Minority Over-sampling Technique

In [7]:
global features_test
global features_train
global target_test
global target_train
print "Percent of Target that is Yes", target_test.mean()

Percent of Target that is Yes 0.075


In [8]:
def confusionMatrixPlot(target_test, target_predicted, target_names):  # target_test, target_predicted_MODEL
    cm = confusion_matrix(target_test, target_predicted)
    plt.matshow(cm)
    plt.title('Confusion matrix')
    plt.colorbar()
    plt.ylabel('True label')
    plt.xlabel('Predicted label') 
    print ('Confusion Matrix Plot:')
    print(cm)

    ### ADD TO MODEL 
    #### scores_SGD = cross_val_score(SGD, features_train, target_train)

    #### print scores_SGD
    #### print scores_SGD.mean()

    accScore = accuracy_score(target_test, 
                     target_predicted)
    print ('Accuracy Score:')
    print (accScore)

    classReport = classification_report(target_test, 
                            target_predicted, 
                            target_names= target_names)

    print('classification report:') 
    print(classReport)
    print ('Confusion Matrix:')
    conMatrix = confusion_matrix(target_test, 
                       target_predicted)
    print (conMatrix)

def CvScores(target_predicted, 
             features_train,  
             target_train, 
             cv):     # target_predicted, features_train,  target_train, cv(int))
    from sklearn.metrics import average_precision_score
    
    scores_rf = cross_val_score(target_predicted,
                                features_train, 
                                target_train, 
                                cv     = cv,
                                scoring='roc_auc',
                                n_jobs = -1)
    print "Cross Validation Score for each K", scores_rf, '\n'
    scores_rf.mean()
    print("ROC AUC: %0.2f (+/- %0.2f)"  % (scores_rf.mean(), scores_rf.std()))

def accScore(clf_y_train_pred, clf_y_test_pred):    
    clf_train = accuracy_score(target_train, clf_y_train_pred) 
    clf_test  = accuracy_score(target_test, clf_y_test_pred) 

    print('train/test accuracies %.3f / %.3f'
      % (clf_train, clf_test))

    'roc_auc'
    
def gridSearch(model, parameters, cv):
    grid = GridSearchCV(model, 
                        parameters,
                        n_jobs = -1, 
                        cv     = cv
                       )
    grid.fit(features_train,
             target_train)
    #target_predicted_SVR = clf_linSVR.predict(features_test)
    grid.fit(features_train, target_train)
    
    print "SCORES",     grid.grid_scores_ , '\n'
    print "BEST Estm",  grid.best_estimator_, '\n'
    print "BEST SCORE", grid.best_score_, '\n'
    print "BEST PARAM", grid.best_params_, '\n' 
    
#CvScores(target_predicted, features_train,  target_train, 2, 'roc_auc')    
#confusionMatrixPlot(target_test, target_predicted_rf, target_names)  


In [9]:
from sklearn.metrics import accuracy_score as accuracy_score
from sklearn.cluster import KMeans as KNN
from sklearn.neighbors import KNeighborsClassifier as KNeighborsClassifier
from sklearn.svm import SVR as SVR 
from sklearn.svm import LinearSVC as LinearSVC
from sklearn.naive_bayes import GaussianNB  as GaussianNB
from sklearn.lda import LDA as LDA
from sklearn.qda import QDA as QDA 
from sklearn.tree import DecisionTreeClassifier as DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier as AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier as RandomForestClassifier
from sklearn.ensemble import BaggingClassifier as BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier as ExtraTreesClassifier  
from sklearn.ensemble import GradientBoostingClassifier as GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier as StochasticGradientDescent  

target_names = ["0 = no", "1 = yes"]
expected = target_test
        
def workerFunction(clf, features_train, features_test, target_train, target_test):
    # Libraries 

    clf = clf
    target_names = ["0 = no", "1 = yes"]
    expected = target_test

    clf.fit(features_train, target_train) 
    
    target_predicted = clf.predict(features_test)
    
    clf_y_train_pred = clf.predict(features_train)
    clf_y_test_pred = clf.predict(features_test)
    
    clf_train = accuracy_score(target_train, clf_y_train_pred) 
    clf_test  = accuracy_score(target_test,  clf_y_test_pred) 
    """
    models_To_Run
    
    models_To_Run()
    
    """
    
    
    global clf_y_train_pred
    global clf_y_test_pred
    global target_predicted
    
    return (clf, target_predicted, clf_y_train_pred, clf_y_test_pred)

# def accScore(clf_y_train_pred, clf_y_test_pred):    
#     clf_train = accuracy_score(target_train, clf_y_train_pred) 
#     clf_test  = accuracy_score(target_test, clf_y_test_pred) 

#     print('train/test accuracies %.3f / %.3f'
#       % (clf_train, clf_test))
    
    
names = [
    'KNN', 
    'DecisionTree',
    'DecisionTree1',
    'RandomForest', 
    'Support Vector Machine (Linear)', 
    'ExtraTrees',
    'GradientBoosting', 
    'StochasticGradientDescent', 
    'MLP',
    'MLP1', 
    'MLP2', 
    'MLP3'
]



classifiers = [
    KNeighborsClassifier(n_neighbors = 4, 
                        algorithm   = 'brute', 
                        leaf_size   = 20, 
                        metric      = 'cosine'),
    
   DecisionTreeClassifier(criterion        = 'gini', 
                          splitter         = 'best',          
                          max_depth        = 1,      
                          min_samples_split= 1, 
                          min_samples_leaf = 1,      
                          min_weight_fraction_leaf = 0.0 , 
                          max_features     = None,   
                          random_state     = None,      
                          max_leaf_nodes   = None,   
                          class_weight     = None,      
                          presort          = False),
    
   DecisionTreeClassifier(criterion    = 'entropy', 
                      splitter         = 'best',          
                      max_depth        = 1,      
                      min_samples_split= 1, 
                      min_samples_leaf = 1,      
                      min_weight_fraction_leaf = 0.0 , 
                      max_features     = None,   
                      random_state     = None,      
                      max_leaf_nodes   = None,   
                      class_weight     = None,      
                      presort          = False),

   RandomForestClassifier(n_estimators      = 500 ,      
                          criterion         = 'gini',
                          max_depth         = None,    
                          min_samples_split = 2,     
                          min_samples_leaf  = 1,       
                          min_weight_fraction_leaf = 0.0, 
                          max_features      = 'auto',  
                          max_leaf_nodes    = None, 
                          bootstrap         = True,    
                          oob_score         = False, 
                          n_jobs            = -1,      
                          random_state      = True , 
                          verbose           = 0 ,      
                          warm_start        = False, 
                          class_weight      = None),
    
   LinearSVC(penalty = 'l2',  
             loss    = 'squared_hinge', 
             dual    =  True, 
             tol     = 0.0001, 
             C       = 1.0,  
             class_weight = 'auto'),

   ExtraTreesClassifier(criterion         = 'gini',  
                        max_depth         =  None,        
                        min_samples_split = 2,       
                        min_samples_leaf  = 1, 
                        min_weight_fraction_leaf = 0.0,     
                        max_features      = 'auto',     
                        max_leaf_nodes    = None,    
                        random_state      = None),

   GradientBoostingClassifier(loss              = 'deviance',  
                              learning_rate     = 0.1,     
                              n_estimators      = 500,           
                              subsample         = 1.0, 
                              min_samples_split = 2,           
                              min_samples_leaf  = 1,
                              min_weight_fraction_leaf = 0.0, 
                              max_depth         = 3,           
                              init              = None,      
                              random_state      = None,        
                              max_features      = None,      
                              verbose           = 0,           
                              max_leaf_nodes    = None,           
                              warm_start        = False,       
                              presort           = 'auto'),

   StochasticGradientDescent(alpha         = 0.0001,  
                             average       = False,     
                             class_weight  = 'balanced', 
                             epsilon       = 0.1,  
                             eta0          = 0.0,       
                             fit_intercept = True, 
                             l1_ratio      = 0.15, 
                             learning_rate = 'optimal', 
                             loss          = 'hinge', 
                             n_iter        = 500,    
                             n_jobs        = -1,        
                             penalty       = 'l1',        
                             power_t       = 0.5,  
                             random_state  = None,      
                             shuffle       = True, 
                             verbose       = 0,   
                             warm_start    = False),
    Classifier(layers=[
        Layer("Rectifier", units = 100),
        Layer("Sigmoid",   units = 100, weight_decay = 0.00001),
        Layer("Softmax")],
                          warning       = None, 
                          weights       = None, 
                          random_state  = None, 
                          learning_rule = u'sgd', 
                          learning_rate = 0.001, 
                          learning_momentum= 0.9, 
                          regularize    = None, 
                          weight_decay  = None, 
                          dropout_rate  = None, 
                          batch_size    = 1, 
                          n_iter        = 100, 
                          n_stable      = 10, 
                          f_stable      = 0.001,
                          valid_set     = None, 
                          valid_size    = 0.0,
                          loss_type     = None,
                          callback      = None, 
                          debug         = False, 
                          verbose       = None),
    Classifier(layers=[
        Layer("Rectifier", units = 100),
        Layer("Sigmoid",   units = 100, weight_decay = 0.001),
        Layer("Softmax")],
                          warning        = None, 
                          weights        = None, 
                          random_state   = None, 
                          learning_rule  = u'sgd', 
                          learning_rate  = 0.01, 
                          learning_momentum= 0.9, 
                          regularize     = None, 
                          weight_decay   = None, 
                          dropout_rate   = None, 
                          batch_size     = 1, 
                          n_iter         = 100, 
                          n_stable       = 10, 
                          f_stable       = 0.001,
                          valid_set      = None, 
                          valid_size     = 0.0,
                          loss_type      = None,
                          callback       = None, 
                          debug          = False, 
                          verbose        = None),
    Classifier(layers=[
        Layer("Rectifier", units = 100),
        Layer("Sigmoid",   units = 100, weight_decay = 0.01),
        Layer("Softmax")],
                          warning         = None, 
                          weights         = None, 
                          random_state    = None, 
                          learning_rule   = u'sgd', 
                          learning_rate   = 0.09, 
                          learning_momentum = 0.05, 
                          regularize      = None,
                          weight_decay    = None, 
                          dropout_rate    = None, 
                          batch_size      = 1, 
                          n_iter          = 100, 
                          n_stable        = 10, 
                          f_stable        = 0.001,
                          valid_set       = None, 
                          valid_size      = 0.0,
                          loss_type       = None,
                          callback        = None, 
                          debug           = False, 
                          verbose         = None),
    Classifier(layers=[
        Layer("Rectifier", units = 100),
        Layer("Sigmoid",   units = 100, weight_decay = 0.001),
        Layer("Softmax")],
                          warning       = None, 
                          weights       = None, 
                          random_state  = None, 
                          learning_rule = u'sgd', 
                          learning_rate = 0.02, 
                          learning_momentum = 0.05, 
                          regularize    = None, 
                          weight_decay  = None, 
                          dropout_rate  = None, 
                          batch_size    = 1, 
                          n_iter        = 100, 
                          n_stable      = 10, 
                          f_stable      = 0.001,
                          valid_set     = None, 
                          valid_size    = 0.0,
                          loss_type     = None,
                          callback      = None, 
                          debug         = False, 
                          verbose       = None)
]


weakNames       = ['Bagging', 'adaBoost']
weakClassifiers = [BaggingClassifier(base_estimator = classifiers,  
                                     n_estimators = 2),
                   
                   AdaBoostClassifier(base_estimator = classifiers,
                                      n_estimators    = 2,
                                      learning_rate   = 1.0,
                                      random_state    = None)
                  ]
ValidationName          = ['ConfusionMatrix', 
                           'CrossValidation Score for n ks', 'Accuracy Score']

ValidationMetricsRunAll = [#confusionMatrixPlot(target_test, target_predicted, target_names),
                           #CvScores(target_predicted, features_train,  target_train, 2),
                           #accScore(clf_y_train_pred, clf_y_test_pred)
                          ]

parameters = []
clf = 0 
import csv 

def strongLearners(names, classifier):
    for name, clf,  in zip(names, classifiers):
        print 'Model Parameters: {}'.format(clf), '\n'
        print 'Name: {}'.format(name), '\n'
        
        workerFunction(clf, 
                       features_train, 
                       features_test, 
                       target_train, 
                       target_test)
        
        accScore(clf_y_train_pred, 
                 clf_y_test_pred)
        
        confusionMatrixPlot(target_test, 
                            target_predicted, 
                            target_names)
        CvScores(clf, 
                 features_train,
                 target_train, 7)
        
        #gridSearch(clf SVC_Param, 4)
                
        
# weakClassifiers = []
# for cls in classifiers:
#     tmp = BaggingClassifier(base_estimator = cls, n_estimators = 2)
#     workerFunction(clf, features_train, features_test, target_train, target_test)
#     print tmp
#     weakLearners.append(tmp)        
  
def weakLearners(weakNames, classifiers):
    weakLearners = []
    """
    Need to add error Error Exception for models that 
    cannot be Bagged or adaBoosted
    """
    for names, clf in zip (weakNames, classifiers):
        weakBagg = BaggingClassifier(base_estimator = clf,
                                     n_estimators = 2
                                    )
        weakAdaB = AdaBoostClassifier(base_estimator  = clf,
                                      n_estimators    = 2,
                                      learning_rate   = 1.0,
                                      random_state    = None
                                     )
    print 'Model Parameters: {}'.format(weakBagg), '\n'
    print 'Name: {}, Bagged'.format(names), '\n'

    workerFunction(weakBagg, 
                   features_train, 
                   features_test,
                   target_train,
                   target_test),
    
    accScore(clf_y_train_pred, 
             clf_y_test_pred),
    
    confusionMatrixPlot(target_test,
                        target_predicted, 
                        target_names), 
    
    CvScores(weakBagg, 
             features_train, 
             target_train, 
             7) 
    
    print 'Model Parameters: {}'.format(weakAdaB), '\n'
    print 'Name: {}, Bagged'.format(names), '\n'

    workerFunction(weakAdaB, 
                   features_train, 
                   features_test,
                   target_train,
                   target_test)
    
    accScore(clf_y_train_pred, 
             clf_y_test_pred)
    
    confusionMatrixPlot(target_test,
                        target_predicted, 
                        target_names)
    
    CvScores(weakAdaB, 
             features_train, 
             target_train, 
             7)

    #weakLearners.append()
        
#########  NEED PARAMETER FUNCTION THAT ######### 
######## RUNS AFTER ALL OF THE BASE MODELS ######### 

def runModels():
    allModels = [strongLearners(names, classifiers),
                 weakLearners(weakNames, classifiers) ]
    for all_ in allModels:
        try:
            funcs()
        except:
            break 
            
            
########## ########## ########## ########## ########## ########## 
   ##### Future Improvements / Implementations ########## #####
########## ########## ########## ########## ########## ########## 

def ParametersFunk(): 
    """
    runs through various params, measures the result, then 
    Prints best to one csv, and others to a second.
    """

# An example that I found on the internet that is applicable to my final product. 
# https://gist.github.com/miguelmalvarez/07e622357b089fee7f21
"""
Logging on event.
"""
def current_timestamp():
    ts = time.time()
    return datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

def log_info(message):
    ts = time.time()
    logging.info(message + " " + current_timestamp())

def init_logging(log_file_path):
    logging.basicConfig(format='%(message)s', level=logging.INFO, filename=log_file_path)

# Magic Sauce. 
"""
"""
def candidate_families():
    
    candidates = [] 
    """
    This will house all models AND PARAMETERS that 
    will be ran throughout this problem space! 
    """
    """
    Example: 
    """
    
    candidates.append(['SVM', SVC(), svm_param ])
    """ 
    Repeat process for all models
    """
    candidates = pd.DataFrame(runModels)
    return candidates
    
def bestModel(): 
    
    def decideMetrics(clf):
        """
        Run logic that pulls metrics that are most applicable 
        to the model that is currently being ran. 
        """
        # Pseudo Code 
        """
        ---------- Measuring of metrics ---------
        If clf is equal to model:
        Then run function associated with that model 
        ifelse second 
        ifelse third 
        ...
        else
        run workerFunction
        """
def saveCSV():
  """
  1. Prepare file for output dump. 
    *** Read in the Metrics that were used to validate each model, 
    *** Then print each Metric and append previous Metric in first column.
  2. For each model ran, create new column (++1)
    *** If model did not contain a Metric, then print, "NaNa For This Model"
  """

# def dataPrep_Train():
#     targetName = ('caption*you')  
#     df = perpareTarget(df, targetName, 'ad.', 'nonad')
#     scaling()

<ipython-input-9-a94bf3579044>:44: SyntaxWarning: name 'clf_y_train_pred' is assigned to before global declaration
  global clf_y_train_pred
<ipython-input-9-a94bf3579044>:45: SyntaxWarning: name 'clf_y_test_pred' is assigned to before global declaration
  global clf_y_test_pred
<ipython-input-9-a94bf3579044>:46: SyntaxWarning: name 'target_predicted' is assigned to before global declaration
  global target_predicted


In [10]:
def main():
    #dataPrep_Train() 
    #runModels()
    strongLearners(names, classifiers)
    #weakLearners(weakNames, classifiers)

In [ ]:
main()

Model Parameters: KNeighborsClassifier(algorithm='brute', leaf_size=20, metric='cosine',
           metric_params=None, n_jobs=1, n_neighbors=4, p=2,
           weights='uniform') 

Name: KNN 

train/test accuracies 0.850 / 0.925
Confusion Matrix Plot:
[[37  0]
 [ 3  0]]
Accuracy Score:
0.925


C:\Users\brandon.tomlin\AppData\Local\Continuum\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


classification report:
             precision    recall  f1-score   support

     0 = no       0.93      1.00      0.96        37
    1 = yes       0.00      0.00      0.00         3

avg / total       0.86      0.93      0.89        40

Confusion Matrix:
[[37  0]
 [ 3  0]]
Cross Validation Score for each K [ 0.1875      0.4375      0.71428571  0.71428571  0.35714286  0.78571429
  0.42857143] 

ROC AUC: 0.52 (+/- 0.21)
Model Parameters: DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=1, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best') 

Name: DecisionTree 

train/test accuracies 0.850 / 0.925
Confusion Matrix Plot:
[[37  0]
 [ 3  0]]
Accuracy Score:
0.925
classification report:
             precision    recall  f1-score   support

     0 = no       0.93      1.00      0.96        37
    1 = yes       0.00      0.00   

C:\Users\brandon.tomlin\AppData\Local\Continuum\Anaconda\lib\site-packages\sklearn\utils\class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


train/test accuracies 0.683 / 0.825
Confusion Matrix Plot:
[[31  6]
 [ 1  2]]
Accuracy Score:
0.825
classification report:
             precision    recall  f1-score   support

     0 = no       0.97      0.84      0.90        37
    1 = yes       0.25      0.67      0.36         3

avg / total       0.91      0.82      0.86        40

Confusion Matrix:
[[31  6]
 [ 1  2]]
Cross Validation Score for each K [ 0.4375      0.6875      0.85714286  0.          1.          0.85714286
  0.28571429] 

ROC AUC: 0.59 (+/- 0.33)
Model Parameters: ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False) 

Name: ExtraTrees 

train/test accuracies 0.983 / 0.950
Confusion Matrix Plot:
[[37  0]
 [ 2  1]]
Accuracy Score:
0.95

c:\users\brandon.tomlin\appdata\local\continuum\anaconda\src\theano\theano\tensor\signal\downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


train/test accuracies 0.850 / 0.925
Confusion Matrix Plot:
[[37  0]
 [ 3  0]]
Accuracy Score:
0.925
classification report:
             precision    recall  f1-score   support

     0 = no       0.93      1.00      0.96        37
    1 = yes       0.00      0.00      0.00         3

avg / total       0.86      0.93      0.89        40

Confusion Matrix:
[[37  0]
 [ 3  0]]


In [ ]:
from scipy.stats import expon

from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline

clf = SVC(C=10.0, 
          kernel='rbf', 
          gamma=0.1, 
          decision_function_shape='ovr')

kernel_svm = Pipeline([('std', StandardScaler()), 
                       ('svc', clf)])

# gridsearch setup
param_grid = [
  {'svc__C': [0.00001, 0.0001, 0.001, 0.01, .1, 1, 10, 100, 1000], 
   'svc__gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1], 
   'svc__kernel': ['rbf', 'linear']},
 ]

gs = GridSearchCV(estimator=kernel_svm, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  n_jobs=-1, 
                  cv=7, 
                  verbose=1, 
                  refit=True,
                  pre_dispatch='2*n_jobs')

gs.fit(features_train, target_train)

print('Best GS Score %.2f' % gs.best_score_)
print('best GS Params %s' % gs.best_params_)


# prediction on the training set
accScore(clf_y_train_pred, 
                 clf_y_test_pred)
confusionMatrixPlot(target_test, 
                    target_predicted, 
                    target_names)
CvScores(clf, 
         features_train,
         target_train, 7)
